In [6]:
# !pip install chromedriver_autoinstaller
# !pip install selenium

In [18]:
import chromedriver_autoinstaller
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.request import Request, urlopen
from selenium.webdriver.chrome.service import Service
from  selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import urllib.request

from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import json

#만개 동안 돌면서 크롤링 해오기

In [8]:
def script_crawling_start():
    text_list = []
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    idx = 20000
    text_container_xpath = '/html/body/div[1]/div[1]/div[4]/div/div[2]/div[1]/div[3]/article/main'
    
    while True:
        try:
                url = f'https://linkareer.com/cover-letter/{idx}?page=1&sort=PASSED_AT&tab=all'
                driver.get(url)
        
                
                text_container = driver.find_element(By.XPATH, text_container_xpath)
                text = text_container.text

                text_list.append(text)
                i += 1
        except:
            if idx > 30000:
                print(text_list)
                break
            else:
                idx += 1
    return text_list

# 함수를 실행시켜 리턴받은 리스트를 csv파일로 만들어주기

In [9]:
# my_series = pd.Series(script_crawling_start())
# my_series.to_csv('./자소서리스트.csv', encoding='utf-8')
# read = pd.read_csv('./자소서리스트.csv', encoding='utf-8')
# read

In [11]:
read = pd.read_csv('./자소서리스트.csv', encoding='utf-8')
read

,Unnamed: 0,0
0,0,"2. 현재 귀하의 가장 부족한 역량은 무엇이며, 역량개발을 위해 어떤 활동과 노력을..."
1,1,1. 삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈\n\n삼성의 핵심 ...
2,2,5. 정직함에 대하여. (경험이 있다면 그 상황에서의 본인의 입장 및 대처 사례)\...
3,3,"3.최근 대형은행의 합병, 인터넷 은행의 등장 등 국내 은행권 경쟁이 심화되고 금융..."
4,4,(My Story)해당회사 및 직무에 지원하는 동기에 대해 기술해 주십시오.\n<G...
...,...,...
5081,5081,의사소통 과정에서 발생하는 본인의 단점 한 가지를 관련 경험과 함께 기술하여 주십시...
5082,5082,1. 보라매병원에 지원하게 된 동기 및 입사 후 실천하고자 하는 목표를 자신의 역량...
5083,5083,"1. 우리 공사에 지원하게 된 이유는 무엇인지 기술하고, 본인의 역량 및 강점을 공..."
5084,5084,1. 수산식품청년마케터에 지원한 동기를 서술하십시오. 700 바이트 공모전 수상 경...


In [19]:
# 전처리
read = read.dropna()

def remove_hanja(text):
    # 한글, 영어, 숫자만 추출하는 정규표현식
    hangul_english_number_pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]+')
    return " ".join(hangul_english_number_pattern.findall(text))

# '자소서' 컬럼의 값들 추출
resumes = read['0']

# 각 텍스트를 처리한 결과를 리스트로 저장
processed_texts = [remove_hanja(text) for text in resumes]

# 새로운 DataFrame 생성
read_df = pd.DataFrame({'Processed_Text': processed_texts})
print(read_df)

                                         Processed_Text
0     2 현재 귀하의 가장 부족한 역량은 무엇이며 역량개발을 위해 어떤 활동과 노력을 하...
1     1 삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈 삼성의 핵심 가치인 ...
2     5 정직함에 대하여 경험이 있다면 그 상황에서의 본인의 입장 및 대처 사례 양심의 ...
3     3 최근 대형은행의 합병 인터넷 은행의 등장 등 국내 은행권 경쟁이 심화되고 금융시...
4     My Story 해당회사 및 직무에 지원하는 동기에 대해 기술해 주십시오 Guide...
...                                                 ...
5077  의사소통 과정에서 발생하는 본인의 단점 한 가지를 관련 경험과 함께 기술하여 주십시...
5078  1 보라매병원에 지원하게 된 동기 및 입사 후 실천하고자 하는 목표를 자신의 역량과...
5079  1 우리 공사에 지원하게 된 이유는 무엇인지 기술하고 본인의 역량 및 강점을 공사의...
5080  1 수산식품청년마케터에 지원한 동기를 서술하십시오 700 바이트 공모전 수상 경험으...
5081  개인소개 및 지원동기 휴먼명조 13포인트 줄간격 160 2페이지 이내 작성 중국어 ...

[5082 rows x 1 columns]


In [20]:
read_df['Processed_Text'][0]

'2 현재 귀하의 가장 부족한 역량은 무엇이며 역량개발을 위해 어떤 활동과 노력을 하고 있는지 기술하시오 필수입력사항 최대 600 최소 100 모두의 소리를 듣겠습니다 음식점에서 아르바이트를 할 당시 중국인 손님을 상대해야 하는일이 생겼습니다 기초 중국어만 가능했던 저는 크게 당황했고 결국영어를 사용해 손님을 상대했습니다 중국인 방문객의 비율이 꾸준히 증가하는 것을 보며 중국어 학습의 필요성을 절감했습니다 부족한 점을 보완하기 위해 저는 중국어 자격증을 취득하는것을 목표로 공부를 시작했습니다 고등학교 시절 배웠던 기본적인 중국어 지식을 바탕으로 학원 수업을빠르게 이해했으며 그 결과 작년 HSK5급에 합격할 수 있었습니다 또한 중국 진출을 꾀하는 기업의 수가 많아 중국어 능력이큰 도움이 될 것이라 판단한 저는 지난 7월 한국외대의외국어연수평가원 과목 중 중국어 과정을 신청했습니다 아침부터 오후까지 중국어에 몰두했으며 2주 동안의 수업과 최종 평가를 통해 저는 최고 성적으로 과정을 수료했습니다 영업관리는 고객 가장 가까이에서 위치한 직무로 지속적인 소통을통해 고객의 목소리에 귀 기울여야 합니다 아직 부족하다고 느껴지는 중국어 회화 능력을 연마해 국내 소비자뿐 아니라 매해 증가하는 중국인 소비자와도 원활한 커뮤니케이션을 이어가겠습니다'

In [14]:
service = Service()    
driver = webdriver.Chrome()
driver.implicitly_wait(1)
url = 'https://www.saramin.co.kr/zf_user/'
driver.get(url)

# 로그인

In [15]:
login_XPATH = '/html/body/header/div[1]/div[2]/div[1]/a[1]'
driver.find_element(By.XPATH, login_XPATH).click()

id = 'auswjqtmxjel'
pw = 'auswjq2@'

id_xpath = '/html/body/div[3]/div/div/div/div/form/div/div/div[2]/div[1]/input'
pw_xpath = '/html/body/div[3]/div/div/div/div/form/div/div/div[2]/div[2]/input'

id_box = driver.find_element(By.XPATH, id_xpath)
pw_box = driver.find_element(By.XPATH, pw_xpath)

id_box.send_keys(id)
pw_box.send_keys(pw)

login_btn_xpath = '/html/body/div[3]/div[1]/div/div/div/form/div/div/div[2]/button'
driver.find_element(By.XPATH, login_btn_xpath).click()

# 이동

In [16]:
question_list = []
strong_text_list = []
def Coverletter(list):
    global question_list, strong_text_list

    for i in range(len(text)):
        ai_url = 'https://www.saramin.co.kr/zf_user/tools/ai-coaching-service'
        driver.get(ai_url)

        text_area_path = '/html/body/div[3]/div[1]/div/div[3]/div/div[1]/div[1]/textarea'
        text_area = driver.find_element(By.XPATH, text_area_path)

        text_area.send_keys(text[i])
        time.sleep(2)

        next_button_path = '/html/body/div[3]/div[1]/div/div[3]/div/div[3]/button[2]'
        driver.find_element(By.XPATH, next_button_path).click()
        
        driver.implicitly_wait(15)
        
        #  질문 받아오는 부분
        q_list = '/html/body/div[3]/div[1]/div/div[3]/div[2]/div[1]/div[2]/ul'
        q_list = driver.find_element(By.XPATH, q_list)
        time.sleep(1)
        q_list = q_list.text.split('\n')
        
        for k in q_list:
            question_list.append(k)
        
        
        # 파란글씨 가져와서 리스트로 만들기
        s_text_xpath = '/html/body/div[3]/div[1]/div/div[3]/div[2]/div[1]/div[1]/div[2]/div/strong'
        try:
            j = 1
            while True:
                q_list_click_xpath = f'/html/body/div[3]/div[1]/div/div[3]/div[2]/div[1]/div[2]/ul/li[{j}]/button/span'
                q_list_click = driver.find_element(By.XPATH, q_list_click_xpath)
                q_list_click.click()
                time.sleep(1)
                strong_text = driver.find_element(By.XPATH, s_text_xpath).text
                strong_text_list.append(strong_text)
                j += 1
        except:
            pass
        print(f'파란글씨: {strong_text_list}\n질문: {question_list}')

# 제이슨

In [21]:
resume_df = pd.DataFrame({'context':read_df['Processed_Text'][0], 'text': [strong_text_list[0:]]})
resume_df

,context,text
0,2 현재 귀하의 가장 부족한 역량은 무엇이며 역량개발을 위해 어떤 활동과 노력을 하...,[]


In [ ]:
# CSV file 출력
resume_df.to_csv('resume_df.csv', index=False, encoding='euc-kr')

# json으로 만들기

In [ ]:
# json 데이터를 형식에 맞게 조정

import json
import pandas as pd

# Read the JSON data using pandas
data_df = pd.read_json('./example.json', encoding='utf-8')

# Convert data_df to the desired JSON format
json_data = {
    "version": "KorQuAD_v1.0_dev",
    "data": []
}

for index, row in resume_df.iterrows():
    context = row["context"]
    text = row["text"]

    # Since 'text' is already a list in the DataFrame, no need to use eval()

    # Append the data to the 'json_data' dictionary
    json_data["data"].append({
        "paragraphs": [
            {
                "qas": [
                    {
                        "answers": [
                            {
                                "text": text,
                                "answer_start": 0
                            }
                        ],
                        "id": f"qas-{index}",
                        "question": "question"  # Replace this with your actual question
                    }
                ],
                "context": context
            }
        ],
        "title": ""
    })

# Save the JSON data to a file
with open('resume_df.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)